
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    `PLEASE ENTER YOUR NAME(S) HERE`

**Student ID(s):**     `PLEASE ENTER YOUR ID(S) HERE`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats
from copy import copy, deepcopy

In [31]:
#    read all datasets 
adult = pd.read_csv("datasets/adult.data", header = None)
bank = pd.read_csv("datasets/bank.data", header = None)
breast_cancer_wisconsin = pd.read_csv("datasets/breast-cancer-wisconsin.data", header = None)
car = pd.read_csv("datasets/car.data", header = None)
lymphography = pd.read_csv("datasets/lymphography.data", header = None)
mushroom = pd.read_csv("datasets/mushroom.data", header = None)
nursery = pd.read_csv("datasets/nursery.data", header = None)
somerville = pd.read_csv("datasets/somerville.data", header = None)
university = pd.read_csv("datasets/university.data", header = None)
wdbc = pd.read_csv("datasets/wdbc.data", header = None)
wine = pd.read_csv("datasets/wine.data", header = None)

In [3]:
#   Helper function that delete all missing values
def delete_missing_value(raw_dataset, missing_values):
    rows = set(raw_dataset[raw_dataset.values == missing_values].index)
    data = raw_dataset.drop(index = rows)
    return data

In [43]:
#    This function should prepare the data by reading it from a file and
#    converting it into a useful format for training and testing

def preprocess(df,missing_v):
#     train=df.sample(frac=0.8,random_state=200)
#     test=df.drop(train.index)
#  Delete all missing values
    train = delete_missing_value(df,missing_v)
    return train

In [49]:
# Global Variable

feature_number = 14
class_column = 14
missing_values = "?"
EPSILON = 1/(2*train_data.shape[0])

# 0 represents nomianl, 1 represents ordinal, 2 represents continuous
feature_types = [2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0]

#  Enter dataset here
train_data = preprocess(adult, missing_values)

# Show some useful information about the data
display(train_data.shape)
display(train_data.head(3))
display(train_data.dtypes)
display(EPSILON)

(30162, 15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


0      int64
1     object
2      int64
3     object
4      int64
5     object
6     object
7     object
8     object
9     object
10     int64
11     int64
12     int64
13    object
14    object
dtype: object

1.657715005636231e-05

In [50]:
#calculate prior
def find_nominal_prob(column):
    prior = {}
    total_number = len(column)
# All possible case
    classes = column.value_counts()
    for key in classes.keys():
        prior[key] = math.log2(classes[key]/total_number)
    return prior

In [51]:
def find_nominal_llh(attributes, llh_dict, data, feature_row):
    all_feature = data[feature_row].value_counts()
    for attribute in attributes:
        feature_dict = {feature_row:{}}
        subset = data[data[class_column] == attribute][feature_row]
        feature_dict[feature_row] = find_nominal_prob(subset)
        for check_feature in all_feature.keys():
            if check_feature not in feature_dict[feature_row].keys():
                feature_dict[feature_row][check_feature] = np.log2(EPSILON)
        if(llh_dict[attribute]):
            llh_dict[attribute].update(feature_dict)
        else:
            llh_dict[attribute] = feature_dict
            
    return llh_dict

In [52]:
def find_numerical_llh(attributes, llh_dict, data, feature):
#    Save mean and Std as llh
    for attribute in attributes:
        feature_dict = {feature:{}}
        subset = subset = data[data[class_column] == attribute][feature]
        mean = np.mean(subset)
        std = np.std(subset)
        feature_dict[feature]["mean"] = mean
        feature_dict[feature]["std"] = std
        if(llh_dict[attribute]):
            llh_dict[attribute].update(feature_dict)
        else:
            llh_dict[attribute] = feature_dict
    return llh_dict

In [53]:
# This function should calculat prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train(data):
    attributes = find_nominal_prob(data[class_column])
    prior = find_nominal_prob(data[class_column])
    llh_dict = dict.fromkeys(attributes)
    for i in range(feature_number):
        if(feature_types[i] == 2):
            find_numerical_llh(attributes, llh_dict, data, i)
        else:
            find_nominal_llh(attributes, llh_dict, data, i)
            
    return prior, llh_dict

prior, llh_dict = train(train_data)

In [19]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def predict(instance, prior, llh_dict):
    all_prob = {}
    for attribute in prior.keys():
        all_prob[attribute] = prior[attribute]
        for i in range(feature_number):
            prob = 0
            if(feature_types[i] == 2):
                mean = llh_dict[attribute][i]["mean"]
                std = llh_dict[attribute][i]["std"]
                prob = stats.norm.pdf(x=instance[i], loc=mean, scale=std)
                if prob > 0.0:
                    prob = np.log2(prob)
                else:
                    prob = np.log2(1 * 10 ** -8)
            else:
                prob = llh_dict[attribute][i][instance[i]]
            all_prob[attribute] += prob
    max_prob = -10000
    max_key = ""
    
    for prob in all_prob.keys():
        if(all_prob[prob] > max_prob):
            max_prob = all_prob[prob]
            max_key = prob
    return max_key

predict(train_data.iloc[100], prior, llh_dict)

'<=50K'

In [20]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(data):
    predict_list = []
    correct = 0
    total = data.shape[0]
    for index, row in data.iterrows():
        if row[14] == predict(row, prior, llh_dict):
            correct += 1
#         result_list = [index,row[14],predict(row, prior, llh_dict)]
#         predict_list.append(result_list)
    correct_rate = correct/total
    return correct_rate
%time
evaluate(train_data)


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.1 µs


0.821928254094556

## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

In [21]:
#给原有的数据做一个备份
backup = train_data.copy()

In [22]:
#Change feature values
feature = train_data.iloc[:,0]
series = feature.copy()
series.sort_values(ascending = False)[0:30000]

19212    90
5104     90
11996    90
1935     90
5406     90
20610    90
24043    90
28463    90
18725    90
15356    90
10210    90
32277    90
222      90
19747    90
18832    90
5272     90
31030    90
6232     90
6624     90
8973     90
14159    90
12975    90
2891     90
1040     90
32367    90
11512    90
15892    90
4070     90
10545    90
5370     90
         ..
29795    17
11571    17
21105    17
26525    17
29982    17
2381     17
21196    17
24535    17
8094     17
28167    17
7360     17
5396     17
26640    17
17486    17
16131    17
26590    17
19173    17
17474    17
17383    17
12971    17
30328    17
5367     17
29130    17
30026    17
27773    17
23881    17
6820     17
10131    17
10128    17
11437    17
Name: 0, Length: 30000, dtype: int64

In [23]:
#找到这30000个数据,并把其的index换为临时index
series[feature.sort_values()[0:30000].index] = 'a'
series.value_counts()

a     30000
90       35
76       22
77       20
80       16
79       15
78       14
81       13
84        8
82        7
83        5
88        3
85        3
86        1
Name: 0, dtype: int64

In [27]:
methods = ['width','freq','quantile']
data_type=[2,0,2,1,1,0,0,0,0,0,2,2,2,0] #data = backup.copy()
def discretise(data,data_type,levels,by):
    '''
    given a dataset and its corresponding data types, discretise all its numerical features into discre te features
by must be chosen from one of 'width' or 'freq', standing for equal-width or equal-frequency
levels takes a postivie integer, deciding the resultant number of feature levels after discretisati on
    '''
    copy_data = data.copy()
    discrete_datatype = data_type.copy()
    
    for i in range(len(data_type)):
        if data_type[i] == 2: # continuous 
            feature = data.iloc[:,i]
            copy_feature = feature.copy()
        # equal width
        if by=='width': 
        # calculate width of each level 
            maximum = np.max(feature)
            minimum = np.min(feature)
            width = (maximum-minimum)/levels # width of each level 
            for j in range(levels): 
                copy_feature[feature<=(maximum-width*j)] = ('level'+str(levels-j)) 
        # equal quantile 
        elif by=='quantile': 
            quantile = 1/levels
            for k in range(levels): 
                copy_feature[feature<=np.quantile(feature,1-quantile*k)] = (‘level'+str(levels-k))
        # equal frequency 
        elif by==‘freq':
            frequency = feature.shape[0]/levels # frequency of each level 
            for m in range(levels): 
                lower = int(frequency*m) # lower bound of index, so has to be integer
                upper = int(frequency*(m+1)) # upper bound of index, so has to be integer 
                copy_feature[feature.sort_values()[lower:upper].index] = ('level' + str(levels-m))

            copy_data.iloc[:,i] = copy_feature 
            discrete_datatype[i] = 1
    return copy_data, discrete_datatype 

width_data,discrete_datatype = discretise(data,data_type,levels=5,by='width')
qtile_data,discrete_datatype = discretise(data,data_type,levels=5,by='quantile')
freq_data,discrete_datatype = discretise(data,data_type,levels=5,by='freq')


SyntaxError: invalid character in identifier (<ipython-input-27-3c95b77d256c>, line 28)

In [28]:
# check one feature
data.iloc[:,10].describe()

NameError: name 'data' is not defined

In [ ]:
width_data.iloc[:,10].value_counts()

In [ ]:
qtile_data.iloc[:,10].value_counts()

In [ ]:
freq_data.iloc[:,10].value_counts()

In [ ]:
width_model = train(width_data,label_index,labels,discrete_datatype,smoothing='epsilon',k=0)
width_preds = predict(width_data,label_index,discrete_datatype,width_model)
width_eval = evaluate(width_preds,labels)

In [ ]:
qtile_model = train(qtile_data,label_index,labels,discrete_datatype,smoothing='epsilon',k=0)
qtile_preds = predict(qtile_data,label_index,discrete_datatype,qtile_model)
qtile_eval = evaluate(qtile_preds,labels)
 

In [ ]:
freq_model = train(freq_data,label_index,labels,discrete_datatype,smoothing='epsilon',k=0)
freq_preds = predict(freq_data,label_index,discrete_datatype,freq_model)
freq_eval = evaluate(freq_preds,labels)

### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

In [ ]:
from random import randint from random import seed
def random_pred(data,label_index): 
    predictions = []
    for i in range(data.shape[0]):
        instance = data.iloc[i,:]
        true_label = instance[label_index]
        seed(i) # different seed for each instance
        index = randint(0,len(labels)-1)
        pred = labels[index]
        predictions.append((i,true_label,pred))
    return predictions
random_predictions = random_pred(data,label_index)
random_eval = evaluate(random_predictions,labels)

In [ ]:
def zero_r_pred(data,label_index):
    majority = data[label_index].value_counts().idxmax()
    predictions = []
    for i in range(data.shape[0]):
        instance = data.iloc[i,:]
        true_label = instance[label_index]
        predictions.append((i,true_label,majority))
    return predictions
zero_r_predictions = zero_r_pred(data,label_index)
zero_r_eval = evaluate(zero_r_predictions,labels)

### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.